In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

class StudentMovementLSTM:
    def __init__(self, input_features=4):
        self.model = self.build_model(input_features)
        self.scaler = MinMaxScaler()
    
    def build_model(self, input_features):
        model = tf.keras.Sequential([
            tf.keras.layers.LSTM(64, 
                input_shape=(None, input_features), 
                return_sequences=True),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.LSTM(32),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        
        model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        return model
    
    def prepare_sequences(self, movement_data, labels):
        """
        Prepare movement sequences for LSTM training
        
        Args:
        - movement_data: List of movement sequences
        - labels: Corresponding truancy labels
        
        Returns:
        - Processed input sequences and labels
        """
        # Normalize sequences
        normalized_sequences = [
            self.scaler.fit_transform(sequence) 
            for sequence in movement_data
        ]
        
        # Pad sequences to consistent length
        padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
            normalized_sequences, 
            padding='post', 
            dtype='float32'
        )
        
        return padded_sequences, np.array(labels)
    
    def train(self, movement_sequences, labels, epochs=50):
        """
        Train LSTM on movement sequences
        
        Args:
        - movement_sequences: List of movement sequences
        - labels: Corresponding truancy labels
        - epochs: Number of training epochs
        
        Returns:
        - Training history
        """
        # Prepare sequences
        X, y = self.prepare_sequences(movement_sequences, labels)
        
        # Split data
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # Train model
        history = self.model.fit(
            X_train, y_train,
            epochs=epochs,
            validation_data=(X_val, y_val),
            batch_size=32
        )
        
        return history
    
    def predict_truancy(self, movement_sequence):
        """
        Predict truancy probability for a movement sequence
        
        Args:
        - movement_sequence: Movement sequence to analyze
        
        Returns:
        - Truancy probability
        """
        # Normalize and pad sequence
        normalized_seq = self.scaler.transform(movement_sequence)
        padded_seq = tf.keras.preprocessing.sequence.pad_sequences(
            [normalized_seq], 
            padding='post', 
            dtype='float32'
        )
        
        # Predict truancy
        truancy_prob = self.model.predict(padded_seq)
        return truancy_prob[0][0]
    
    def save_model(self, filepath='student_movement_lstm.h5'):
        """Save trained model"""
        self.model.save(filepath)
    
    def load_model(self, filepath='student_movement_lstm.h5'):
        """Load pre-trained model"""
        self.model = tf.keras.models.load_model(filepath)

# Example dataset generation
def generate_synthetic_dataset(num_sequences=1000):
    """
    Generate synthetic movement sequences for training
    
    Returns:
    - movement_sequences: List of movement sequences
    - labels: Corresponding truancy labels
    """
    movement_sequences = []
    labels = []
    
    for _ in range(num_sequences):
        # Simulate different movement patterns
        seq_length = np.random.randint(10, 50)
        movement_seq = np.random.rand(seq_length, 4)
        
        # Randomly assign truancy label
        truancy_label = np.random.choice([0, 1], p=[0.7, 0.3])
        
        # Modify sequence based on truancy
        if truancy_label == 1:
            # Irregular movement patterns for truancy
            movement_seq[:, 0] *= 2  # Amplify x-coordinate variations
            movement_seq[:, 1] += np.random.rand(seq_length)  # Add noise
        
        movement_sequences.append(movement_seq)
        labels.append(truancy_label)
    
    return movement_sequences, labels

# Training pipeline
def train_student_movement_model():
    # Generate synthetic dataset
    movement_sequences, labels = generate_synthetic_dataset()
    
    # Initialize and train LSTM model
    lstm_model = StudentMovementLSTM()
    history = lstm_model.train(movement_sequences, labels)
    
    # Save trained model
    lstm_model.save_model()
    
    return lstm_model

# Execute training
trained_model = train_student_movement_model()